<a href="https://colab.research.google.com/github/Venky90505/Text-to-image-using-stable-diffusion/blob/main/Our_Project_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch

In [ ]:
# Model setup
model_id = "stabilityai/stable-diffusion-2"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    scheduler=scheduler,
    revision="fp16",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

In [ ]:
prompt = "realistic photo of 3d cartoon style of a cat in a space suit with milky way background"

In [ ]:
def txt2img(prompt):

  image = pipe(prompt, height=768, width=768, guidance_scale = 10).images[0]

  image.save("sd_image.png")

  return image

In [ ]:
txt2img(prompt)

In [ ]:
pip install gradio

In [ ]:
# Image generation function
def generate_images(prompt, num_images=1):
    """Generate images based on text prompt."""
    try:
        image_paths = []
        for i in range(int(num_images)):
            image = pipe(prompt, height=768, width=768, guidance_scale=10).images[0]
            file_path = f"generated_image_{i + 1}.png"
            image.save(file_path)
            image_paths.append(file_path)
        return image_paths
    except Exception as e:
        return f"Error generating images: {str(e)}"

In [ ]:
# Example prompts
example_prompts = [
    ["realistic photo of 3d cartoon style of a cat in a space suit with milky way background", 1],
    ["futuristic cyberpunk cityscape at night with neon signs", 2],
    ["serene mountain landscape with crystal clear lake", 1]
]

In [ ]:
# Custom CSS and JavaScript
custom_script = """
<style>
body {
    margin: 0;
    font-family: 'Helvetica Neue', Arial, sans-serif;
    background: #0A0A0B !important;
    color: #E0E0E0 !important;
    height: 100vh;
    overflow-x: hidden;
}
.container {
    max-width: 1280px;
    margin: 20px auto;
    background: #14141E !important;
    border-radius: 12px;
    padding: 30px;
    box-shadow: 0 8px 24px rgba(0, 0, 0, 0.5);
    border: 1px solid #2A2A3A;
}
h1 {
    color: #FEBE10 !important;
    text-align: center;
    font-size: 2.2em;
    margin-bottom: 20px;
    letter-spacing: 1px;
    text-transform: uppercase;
}
.input-box textarea {
    background: #1E1E2A !important;
    border: 1px solid #404060 !important;
    border-radius: 8px;
    padding: 12px;
    color: #E0E0E0 !important;
    font-size: 1em;
    transition: border-color 0.3s, box-shadow 0.3s;
}
.input-box textarea:focus {
    border-color: #FFD700 !important;
    box-shadow: 0 0 8px rgba(255, 215, 0, 0.3) !important;
}
.slider {
    --track-color: #404060 !important;
    --thumb-color: #FFD700 !important;
}
button.button {
    background: linear-gradient(90deg, rgba(0,255,252,1) 0%, rgba(10,154,219,1) 12%, rgba(112,100,225,1) 52%, rgba(255,0,237,1) 100%) !important;
    border: none !important;
    padding: 12px 24px !important;
    border-radius: 8px !important;
    color: #0A0A0B !important;
    font-weight: 600 !important;
    text-transform: uppercase !important;
    letter-spacing: 1px !important;
    cursor: pointer !important;
    transition: all 0.3s ease !important;
}
button.button:hover {
    background: rgba(0, 0, 0, 0.7) !important; /* hover */
    box-shadow: 0 0 12px rgba(255, 215, 0, 0.5) !important;
    color: #E0E0E0 !important;
}
.gallery {
    background: #1E1E2A !important;
    border-radius: 10px;
    padding: 15px;
    border: 1px solid #404060;
    margin-top: 20px;
}
.gallery img {
    border-radius: 8px !important;
    border: 1px solid #404060 !important;
    transition: all 0.3s ease !important;
}
.browser-default .gallery img:hover {
    border-color: #FFD700 !important;
    box-shadow: 0 0 12px rgba(255, 215, 0, 0.2) !important;
}
.status-text {
    color: #FFD700 !important;
    text-align: center;
    font-size: 0.9em;
    margin-top: 20px;
}
</style>

<script>
document.addEventListener('DOMContentLoaded', () => {
    const button = document.querySelector('button.button');
    if (button) {
        button.addEventListener('click', () => {
            button.style.transform = 'scale(0.98)';
            setTimeout(() => {
                button.style.transform = 'scale(1)';
            }, 100);
            const status = document.querySelector('.status-text');
            if (status) status.textContent = 'Processing...';
        });
    }

    const input = document.querySelector('.input-box textarea');
    if (input) {
        input.addEventListener('input', (e) => {
            if (e.target.value.length > 0) {
                input.style.boxShadow = '0 0 8px rgba(255, 215, 0, 0.3)';
            } else {
                input.style.boxShadow = 'none';
            }
        });
    }

    const observer = new IntersectionObserver((entries) => {
        entries.forEach(entry => {
            if (entry.isIntersecting) {
                entry.target.style.opacity = 0;
                entry.target.style.transition = 'opacity 0.5s ease';
                setTimeout(() => {
                    entry.target.style.opacity = 1;
                }, 100);
                observer.unobserve(entry.target);
            }
        });
    }, { threshold: 0.1 });

    document.querySelectorAll('.gallery img').forEach(img => {
        observer.observe(img);
    });
});
</script>
"""

In [ ]:
# Gradio interface
import gradio as gr
with gr.Blocks(title="AI Image Generation") as demo:
    gr.HTML(f"""
        <div>
            {custom_script}
            <div class="container">
                <h1>Prompt to Image Generator</h1>
    """)


    # Prompt and results row at the top
    with gr.Row():
        with gr.Column(scale=1):
            prompt_input = gr.Textbox(
                label="Prompt",
                placeholder="Enter your Imagination...",
                elem_classes="input-box"
            )
            num_images_input = gr.Slider(
                minimum=1,
                maximum=5,
                step=1,
                value=1,
                label="Image Count",
                elem_classes="slider"
            )
            generate_btn = gr.Button(
                "Generate",
                elem_classes="button"
            )
        with gr.Column(scale=2):
            output_gallery = gr.Gallery(
                label="Generated Images",
                elem_classes="gallery",
                columns=2,
                object_fit="cover",
                height="auto"
            )

    gr.HTML('<div class="status-text"></div>')

    gr.Examples(
        examples=example_prompts,
        inputs=[prompt_input, num_images_input],
        outputs=output_gallery,
        fn=generate_images,
        cache_examples=False,
        label="Example Prompts"
    )

    generate_btn.click(
        fn=generate_images,
        inputs=[prompt_input, num_images_input],
        outputs=output_gallery
    ).then(
        fn=lambda: gr.update(value="Ready"),
        inputs=None,
        outputs=gr.HTML(elem_classes="status-text")
    )

    gr.HTML("</div></div>")

In [ ]:
# Launch the interface
demo.launch(share=True, debug=True)